Laad nuttige functionaliteit

In [ ]:
# Creeren van figuren 
using Plots, LaTeXStrings
plot_options = (legend=false,linewidth=5)

# Gebruik van widget met slider (als het toeval het toestaat)
using Interact 

# Verzameling van bases 
using BasisFunctions 

Basisfunctions.jl is een pakket van onze onderzoeksgroep en bevat functionaliteit voor het benaderen van een functie met een verzameling van gegeven functies

# I. Functiebenadering op een interval met veeltermen

## I.1. Monomialen

We kunnen functies benaderen met bijvoorbeeld een set in de vorm 

$$ 1,\quad x,\quad x^2,\quad x^3,\quad \dots $$

We benaderen dus $f(x)$ met 
$$ c_0 + c_1x+c_2x^2 +\cdots + c_Nx^N.$$
Of in het kort 
$$ f(x)\approx \sum_{k=0}^N c_kx^k.$$

De onbekenden zijn 
$$ c_0,\quad c_1,\quad c_2,\quad \cdots,\quad c_N$$

en kunnen gevonden worden door het oplossen van een $N\times N$matrix 


$$\begin{bmatrix}  
1 & x_1 & x_1^2 & \cdots & x_1^N \\1 & x_2 & x_2^2 & \cdots & x_2^N \\\vdots & \vdots & \vdots & \ddots & \vdots \\1 & x_N & x_N^2 & \cdots & x_N^N \\\end{bmatrix} \begin{bmatrix}  c_0 \\c_1 \\\vdots \\c_N \\\end{bmatrix} = \begin{bmatrix} f(x_1) \\ f(x_2) \\ \vdots \\f(x_N)\end{bmatrix}.$$

In [ ]:
N = 10
basis = Monomials(N+1)
gr();@manipulate for i in 1:length(basis) 
    plot(x->basis[i](x), -1,1;size=(300,300),ylims=(-1,1),plot_options...,title=latexstring("x^$(i-1)"))
end

Dit is echter een <i><b>onstabiele</b></u> basis (set van functies) omdat de verschillende functies goed op elkaar beginnen lijken.

Vergelijk bijvoorbeel $x^8$ en $x^{10}$:

In [ ]:
basis = Monomials(N+1)
gr();plot(x->basis[10](x), -1,1;ylims=(0,1),plot_options...)
plot!(x->basis[8](x), -1,1;size=(300,300),ylims=(0,1),plot_options...)

## I.2. Chebyshev

Een Chebyshev basis bevat ook veeltermen, maar is <b>stabieler</b>:

$$ T_0(x) = 1, \quad T_1(x) = x,\quad T_2(x) = 2x^2-1, \quad T_3(x) = 4x^3-3x,\quad\dots $$ 

In [ ]:
basis = ChebyshevT(N);
gr();@manipulate for k in 1:length(basis)
    plot(x->basis[k](x), -1,1;size=(300,300),ylims=(-1,1),plot_options...,title=latexstring("T_$(k-1)(x)"))
end

## I.2.a. Functiebenadering op een interval 

Stel dat we een exponentiele functie $f(x)=e^x$, zie hieronder, willen benaderen in $[-1,1]$

In [ ]:
line_fun = exp
plot(line_fun,-1,1,legend=false,size=(300,200),linewidth=10,background_color = :transparent,foreground_color=:black)
# savefig("interval")

Dan zoeken we getallen $c_0, c_1,\dots, c_N$ zodat 
$$ e^x≈\sum_{k=0}^N c_kT_k(x) $$.

In [ ]:
gr();@manipulate for n in 1:21
    fun = approximate(ChebyshevT(n),line_fun)
    plot(fun;size=(1000,200),layout=(1,3),plot_options...,title="Benadering")
    plot!(line_fun,-1,1;subplot=1,color="black")
    plot!(fun,line_fun;subplot=2,ylims=(1e-16,1),plot_options...,title="Fout")
    scatter!(abs.(coefficients(fun));ylims=(1e-18,10),yscale=:log10,subplot=3,title=L"$c_k$",plot_options...)
end

-  $n=1$: We gebruiken slechts 1 functie om $e^x$ te benaderen. De fout is overal groot. 
-  $n=5$: Visueel is de benadering in orde. De fout is $\approx 10^{-2}$. 
-  $n=14$: De fout is bereikt <b>machineprecisie</b> ($\approx 10^{-16}$). Patroon van dalende coefficienten zichtbaar. 
-  $n=17$: De coefficienten bereiken minimum en blijven klein.

# II. Functiebenadering op een cirkel met Fourier basis

## II.1. Fourier basis

Een Fourier basis bevat cosinussen en sinussen: 
$$ 1, \quad\cos(\pi x), \quad \sin (\pi x),\quad \cos(2\pi x),\quad \sin (2\pi x), \dots$$

In [ ]:
basis = Fourier(N+1);
gr();@manipulate for i in 1:length(basis) 
    plot(basis[i];size=(600,200),ylims=(-1,1),plot_complex=true,plot_options...)
end

Deze basis is <b>periodisch</b>: je kan de functies achter elkaar plakken en het resultaat blijft zacht verlopen. 
Daarom is deze basis geschikt om functies op een <b>cirkel</b> te benaderen.

In [ ]:
T = ChebyshevT(N+1);F = Fourier(N+1);

gr();@manipulate for i in 1:N>>1 
    plot(F[i];size=(1000,300),layout=(1,4),ylims=(-1.5,1.5),plot_options...)
    scatter!([1.],[real(F[i](1.))];plot_options...,c=:lightgreen,subplot=1,y_ticks=false,markersize=5,title="")
    plot!(F[i];subplot=2,ylims=(-1.5,1.5),plot_complex=false,yaxis=false,plot_options...)
    scatter!([0.],real([F[i](1.)]),c=:lightgreen,subplot=2,markersize=5)
    plot!(T[i];subplot=3,ylims=(-1.5,1.5),plot_options...)
    scatter!([1.],[T[i](1.)],c=:red,subplot=3,markersize=5)
    plot!(T[i];subplot=4,ylims=(-1.5,1.5),yaxis=false,plot_options...)
    scatter!([-1.],[T[i](-1.)],c=:red,subplot=4,markersize=5)
end

### II.1.a. Functiebenadering op een cirkel

In [ ]:
θs = LinRange(0,1,500)
zs = cis.(2pi.*θs)
xs = real.(zs)
ys = imag.(zs)
gr();plot3d(xs,ys,zeros(length(θs));size=(300,200),plot_options...)

In [ ]:
cirkel_f = θ->exp(cos(6π*θ))-.7
zs = cirkel_f.(θs)
gr();
plot3d(xs,ys,zeros(length(θs));size=(300,200),c=:black,background_color = :transparent,foreground_color=:black);
plot3d!(xs,ys,zs;size=(300,200),color=cgrad(:redsblues)[zs],plot_options...,
        xlabel="x",ylabel="y",zlabel="z")
# savefig("cirkel")

In [ ]:
gr();@manipulate throttle = 0.1  for n in 1:2:101
    fun = approximate(Fourier(n),cirkel_f)
    zs = real.(fun.(θs))
    plot3d(xs,ys,zs;color=cgrad(:redsblues)[zs],layout=(1,3),size=(1000,300),plot_options...,title="Benadering")
    plot3d!(xs,ys,cirkel_f.(θs);subplot=1,color="black")
    plot!(fun,cirkel_f;ylims=(1e-16,1),subplot=2,title="Fout",xlabel="θ",ylabel="z")
    scatter!(abs.(coefficients(fun));ylims=(1e-18,10),yscale=:log10,subplot=3,title=L"$c_k$",
            xlabel="k",ylabel="",plot_options...)
end

# III. Functiebenadering op een vierkant

## III.1. 2-dimensionele Chebyshevbasis

In [ ]:
basis = ChebyshevT(3)^2;
plot(basis[9];c=:redsblues,zlims=(-1.5,1.5),size=(300,200),cbar=false,background_color = :transparent,foreground_color=:black)
# savefig("vierkant")
gr();@manipulate throttle = 0.1  for k in 1:length(basis)
    plot(basis[k];c=:redsblues,zlims=(-1.5,1.5),size=(300,300),cbar=false)
end

## III.1. 2-dimensionele splinebasis

In [ ]:
using CompactTranslatesDict: BSplineTranslatesBasis
basis = BSplineTranslatesBasis(4,1)^2;
gr();@manipulate throttle = 0.1  for k in 1:length(basis)
    plot(basis[k];c=:redsblues,layout=(1,2),cbar=false)
    heatmap!(basis[k];subplot=2,c=:redsblues,cbar=false,ratio=1,size=(600,300))
end

# IV. Functiebenadering op een complexe geometrie?

In [ ]:
using FrameFun
using FrameFunApplications.SplineCircleExample: domein
using FrameFunApplications.PDEPlots:heatmap_matrix
D2_fun(x,y) = (y+.5)*cos(x)
D2_fun((x,y),) = D2_fun(x,y)
D2_fun(x::AbstractGrid) = D2_fun.(x)

plot(domein(r=.15);size=(600,300),layout=(1,2),xlims=(0,1),ylims=(0,1))
plot!(mandelbrot(),subplot=2,xlims=(-1,.5),ylims=(-.5,.5))

#### Strategie: plaats complexe geometrie in een eenvoudige box, hier een rechthoek

## IV.1. Chebyshev basis op $[-1,1/2]×[-1/2,1/2]$ $\rightarrow$ Chebyshev frame op geometrie

In [ ]:
(x,y),M=heatmap_matrix(D2_fun,mandelbrot(),EquispacedGrid(101,-.5,1)×EquispacedGrid(101,-.5,.5))
surface(x,y,M;c=:redsblues,size=(300,200),cbar=false,xlims=(-.5,1),ylims=(-.5,.5),xticks=[-.5,.5],yticks=[-.5,.5],
        background_color = :transparent,foreground_color=:black)
# savefig("mandelbrot")

In [ ]:
frame = platform(extensionframe((ChebyshevT(10)→(-1.0..0.5))⊗(ChebyshevT(10)→(-.5..0.5)),mandelbrot()))

gr();@manipulate throttle = 0.1  for k in 1:12
    F,A,b,c,_=approximate(D2_fun,frame,(k,k));
    plot(;layout=(1,3))
    heatmap!(F;subplot=1,c=:redsblues)
    plot!(F,D2_fun;subplot=2,c=:redsblues,clims=(-16,0))
    scatter!(sort(abs.(c)[:];rev=true);yscale=:log10,ylims=(1e-16,10),c=:redsblues,subplot=3,
            size=(1000,300),legend=false)
end


Observaties 
- Fout daalt naar machineprecisie 
- Grootte van coefficienten bereiken niet langer machineprecisie

## IV.2. Splinebasis op $[0,1]^2$ $\rightarrow$ spline frame op geometrie

In [ ]:
(x,y),M=heatmap_matrix(D2_fun,domein(r=.15),EquispacedGrid(101,0,1)^2)
surface(x,y,M;c=:redsblues,size=(300,200),cbar=false,xlims=(-.5,1),xticks=[0,.5],yticks=[0,.5])

In [ ]:
frame = platform(extensionframe(BSplineTranslatesBasis(10,3)^2,domein(r=.15)));
F,A,b,c,_=approximate(D2_fun,frame,(10,10));
plot(;layout=(1,3))
heatmap!(F;subplot=1)
plot!(F,D2_fun;subplot=2)
scatter!(abs.(c)[:];subplot=3,size=(1000,300))

gr();@manipulate throttle = 0.1  for k in 5:30
    F,A,b,c,_=approximate(D2_fun,frame,(k,k));
    plot(;layout=(1,3))
    heatmap!(F;subplot=1,c=:redsblues)
    plot!(F,D2_fun;subplot=2,c=:redsblues,clims=(-16,0))
    scatter!(sort(abs.(c)[:];rev=true);yscale=:log10,ylims=(1e-16,10),c=:redsblues,subplot=3,
            size=(1000,300),legend=false)
end

Observaties
- Fout daalt trager dan bij Chebyshev
- Veel coefficienten zijn nul: dit zijn de coefficienten met spline die nul is in geometrie. 